In [1]:
using DataFrames, DataFramesMeta, Optim, Revise, Serialization, DebuggingUtilities, LinearAlgebra, Statistics
using JuMP, Ipopt

In [12]:
const datadir = "/export/storage_adgandhi/MiscLi/factract";
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule; const m = BLPmodule;

In [13]:
testmodes = ["full", "fl", "fl17"]
testmode = testmodes[2]################# EDIT THIS ##################

ec = deserialize("$datadir/analysis/ec_nlls$testmode.jls");

---
### jump

In [14]:
q_obs = ec.q_obs;
tracts = ec.tracts;
n_firms = length(ec.firms);
X = [t.X[:,1:2] for t in tracts];
D = [t.D[:,1:2] for t in tracts];

In [15]:
j_indexer, t_indexer, positionin = m.create_indices(tracts, n_firms);

In [16]:
θx, θd, share, mktq, firmq = m.run_nlls(X, D, tracts, n_firms, q_obs);


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality co

In [24]:
ηDj = m.compute_elasticity(D, θd, share, mktq, t_indexer, positionin, n_firms);
mean(ηDj)

-2.744910976619643

In [33]:
X_dc = [t.X[:,[1]] for t in tracts];
X_rn = [t.X[:,[2]] for t in tracts];

In [31]:
η_dc = m.compute_elasticity(X_dc, [θx[1]], share, mktq, t_indexer, positionin, n_firms);
mean(η_dc)

-2.4279600170299322

In [35]:
η_rn = m.compute_elasticity(X_rn, [θx[2]], share, mktq, t_indexer, positionin, n_firms);
mean(η_rn)

-0.05438817773708475